In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from tqdm.notebook import tqdm
import requests, cma

import pickle, os

In [2]:
reference_path = "../../../results/road/freeflow/reference_api.parquet"
output_path = "../../../results/road/freeflow/calibration_cache_api.pickle"

# reference_path = "../../../results/road/freeflow/reference_survey.parquet"
# output_path = "../../../results/road/freeflow/calibration_cache_survey.pickle"

routing_endpoint = "http://localhost:8054/router/road"
departure_time = 4 * 3600
maximum_batch_size = 400

In [3]:
# Load reference
df_reference = pd.read_parquet(reference_path)

In [4]:
# Prepare requests
df_reference["request_index"] = np.arange(len(df_reference))

# Convert to requests
request_list = []

for index, row in df_reference.iterrows():
    request_list.append({
        "request_index": int(row["request_index"]),
        "origin_x": row["origin_x"],
        "origin_y": row["origin_y"],
        "destination_x": row["destination_x"],
        "destination_y": row["destination_y"],
        "departure_time_s": departure_time,
        "consider_parallel_links": True
    })

In [5]:
# Prepare querying
def query_requests(request_list, settings):
    df_response = []
    batch_index = 0

    while batch_index * maximum_batch_size < len(request_list):
        batch = request_list[batch_index * maximum_batch_size : (batch_index + 1) * maximum_batch_size]

        response = requests.post(routing_endpoint, json = {
            "batch": batch,
            "freespeed": settings
        })

        df_response.append(pd.DataFrame.from_records(response.json()))
        batch_index += 1

    return pd.concat(df_response)

In [6]:
# Define calibration variables
variables = [
    { "name": "major_factor", "initial": 1.0, "bounds": (1.0, np.inf) },
    { "name": "intermediate_factor", "initial": 1.0, "bounds": (1.0, np.inf) },
    { "name": "minor_factor", "initial": 1.0, "bounds": (1.0, np.inf) },
    { "name": "major_crossing_penalty_s", "initial": 0.0, "bounds": (0.0, np.inf) },
    { "name": "minor_crossing_penalty_s", "initial": 0.0, "bounds": (0.0, np.inf) },
]

In [7]:
# Extend with index information for CMA-ES evaluation
variables_map = { v["name"]: v for v in variables }

active_index = 0

# First find active variables
for variable in variables:
    variables_map[variable["name"]] = variable
    variable["index"] = active_index
    active_index += 1

In [8]:
# Define the optimization objective
def calculate_objective(df_evaluation, df_reference):
    df_reference = df_reference[["request_index", "reference_travel_time_s", "weight"]].copy()

    df_evaluation = df_evaluation[["request_index", "total_travel_time_min"]].copy()
    df_evaluation["evaluation_travel_time_s"] = df_evaluation["total_travel_time_min"] * 60.0
    df_evaluation = df_evaluation[["request_index", "evaluation_travel_time_s"]]

    df_comparison = pd.merge(df_reference, df_evaluation, on = "request_index")
    df_comparison["difference_s"] = np.abs(df_comparison["evaluation_travel_time_s"] - df_comparison["reference_travel_time_s"])

    if False:
        mean = np.sum(df_comparison["reference_travel_time_s"] * df_comparison["weight"]) / df_comparison["weight"].sum()
        ss_residuals = np.sum(df_comparison["weight"] * df_comparison["difference_s"]**2)
        ss_total = np.sum(df_comparison["weight"] * (df_comparison["reference_travel_time_s"] - mean)**2)
        R2 = 1 - ss_residuals / ss_total
        
        return (
            1 - R2, df_comparison
        )
    
    return (
        np.sum(df_comparison["weight"] * np.abs(df_comparison["difference_s"])) / df_comparison["weight"].sum(), 
        df_comparison
    )

In [9]:
# Prepare function to convert CMA-ES' candidate to freespeed settings
def prepare_settings(values):
    settings = {}
    
    for variable in variables:
        settings[variable["name"]] = values[variable["index"]]
    
    return settings

In [10]:
# Prepare bounds and initial values
initial = []
bounds = [[], []]

for variable in variables:
    initial.append(variable["initial"])
    bounds[0].append(variable["bounds"][0])
    bounds[1].append(variable["bounds"][1])

In [11]:
# Test connection
assert len(query_requests(request_list[:5], prepare_settings(initial))) == 5

In [12]:
# Configure CMA-ES
seed = 1000
sigma = 1.0
iterations = 200

options = cma.CMAOptions()
options.set("bounds", bounds)
options.set("seed", seed)

algorithm = cma.CMAEvolutionStrategy(initial, sigma, options)

# Load cached data for previous iterations
history = []

if os.path.exists(output_path):
    with open(output_path, "rb") as f:
        history = pickle.load(f)

        algorithm.feed_for_resume(
            [h["candidate"] for h in history[1:]], # first one is initial
            [h["objective"] for h in history[1:]]
        )

# Perform a new batch of iterations
for iteration in range(iterations):
    initial_evaluation = len(history) == 0
    candidates = [initial]

    if not initial_evaluation:
        candidates = algorithm.ask()

    objectives = []

    for candidate in candidates:
        settings = prepare_settings(candidate)
        df_response = query_requests(request_list, settings)
        objective, df_comparison = calculate_objective(df_response, df_reference)

        objectives.append(objective)

        history.append({
            "candidate": candidate,
            "settings": settings,
            "objective": objective,
            "evaluation": df_comparison,
            "initial": initial_evaluation
        })

    if not initial_evaluation:
        algorithm.tell(candidates, objectives)
        algorithm.disp()

    # Save after a successful CMA-ES iteration
    with open(output_path, "wb+") as f:
        pickle.dump(history, f)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=1000, Tue Aug 27 23:10:40 2024)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 1.439493056314596e+02 1.0e+00 8.36e-01  8e-01  8e-01 11:28.6
    2     16 2.055396397765077e+02 1.3e+00 7.73e-01  7e-01  8e-01 24:33.8
    3     24 2.590443594457029e+02 1.4e+00 6.77e-01  6e-01  7e-01 37:43.6
    4     32 2.035173255678356e+02 1.6e+00 5.75e-01  5e-01  6e-01 45:38.9
    5     40 1.577783506556885e+02 1.6e+00 5.22e-01  4e-01  5e-01 52:38.6
    6     48 1.597830361844063e+02 2.0e+00 4.27e-01  3e-01  4e-01 58:57.0
    7     56 1.643791383083877e+02 1.9e+00 3.70e-01  2e-01  4e-01 66:09.7
    8     64 1.555278434581125e+02 2.0e+00 3.25e-01  2e-01  3e-01 70:44.8
    9     72 1.384091781283676e+02 2.0e+00 2.96e-01  2e-01  3e-01 75:44.5
   10     80 1.341143278906355e+02 2.0e+00 3.15e-01  2e-01  3e-01 79:11.1
   11     88 1.356750792696052e+02 1.9e+00 3.28e-01  2e-01  3e-01 83:16.1
   12     96 1.36623611057

KeyboardInterrupt: 